In [1]:
import torch
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [2]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader

In [3]:
torch.manual_seed(0)

transf = tr.Compose(
    [tr.ToTensor(),
     tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data/', train=False, download=True, transform=transf)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 *5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
for epoch in range(30):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data

        # 변화도(Gradient) 매개변수를 0으로 만들고
        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.213
[1,  4000] loss: 1.870
[1,  6000] loss: 1.705
[1,  8000] loss: 1.587
[1, 10000] loss: 1.528
[1, 12000] loss: 1.482
[2,  2000] loss: 1.416
[2,  4000] loss: 1.387
[2,  6000] loss: 1.374
[2,  8000] loss: 1.350
[2, 10000] loss: 1.322
[2, 12000] loss: 1.303
[3,  2000] loss: 1.241
[3,  4000] loss: 1.248
[3,  6000] loss: 1.240
[3,  8000] loss: 1.218
[3, 10000] loss: 1.216
[3, 12000] loss: 1.205
[4,  2000] loss: 1.114
[4,  4000] loss: 1.151
[4,  6000] loss: 1.153
[4,  8000] loss: 1.134
[4, 10000] loss: 1.141
[4, 12000] loss: 1.133
[5,  2000] loss: 1.057
[5,  4000] loss: 1.061
[5,  6000] loss: 1.076
[5,  8000] loss: 1.076
[5, 10000] loss: 1.065
[5, 12000] loss: 1.094
[6,  2000] loss: 0.981
[6,  4000] loss: 1.019
[6,  6000] loss: 1.009
[6,  8000] loss: 1.021
[6, 10000] loss: 1.039
[6, 12000] loss: 1.024
[7,  2000] loss: 0.940
[7,  4000] loss: 0.943
[7,  6000] loss: 0.963
[7,  8000] loss: 0.985
[7, 10000] loss: 1.016
[7, 12000] loss: 0.977
[8,  2000] loss: 0.890
[8,  4000] 

In [7]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [8]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [9]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Predicted:  plane truck deer  deer 


In [10]:
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = net(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 58 %


In [11]:
# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 78.4 %
Accuracy for class: car   is 69.8 %
Accuracy for class: bird  is 48.5 %
Accuracy for class: cat   is 37.1 %
Accuracy for class: deer  is 50.3 %
Accuracy for class: dog   is 52.4 %
Accuracy for class: frog  is 64.9 %
Accuracy for class: horse is 58.2 %
Accuracy for class: ship  is 64.2 %
Accuracy for class: truck is 61.5 %


# MobileNet Code 쳐보면서 궁금한거 만들기

In [1]:
# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch import optim

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as tr
from torch.utils.data import Dataset, DataLoader
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
import time
import copy

# FER Summary
from skimage import io, transform
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import re
import PIL

# random seed fixed
''' random seed fixed'''
import random
import torch.backends.cudnn as cudnn

path = "/home/sldev1/Project/hyeongeun_test/data/FER"
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [2]:
from torch.utils.data import Dataset
import glob


def img_load(img_path):

    jpg = glob.glob(img_path+'*.jpg')
    sort_jpg = sorted(jpg, key=lambda s: int(re.findall(r'\d+', s)[1]))
    #print("img_load def is play")
    return sort_jpg


def label_load(label_path):
    #enumerate #파이썬내장함수.. 강민규사원님은 이걸로 쓰셨는데 느리더래
    label = glob.glob(label_path+'*exp.npy')
    sort_label = sorted(label,key=lambda s: int(re.findall(r'\d+', s)[1]))
    #print("label_load def is play")
    return sort_label
    
    
    
class MyFERDataset(Dataset):
    
    def __init__(self, img_path_base, label_path_base, train=None, transform=None):
        super(MyFERDataset, self).__init__()
        
        if train == True:
            self.img_path = img_path_base+'/train_set/images/'
            self.label_path = label_path_base+'/train_set/annotations/'
        else:
            self.img_path = img_path_base+'/val_set/images/'
            self.label_path = label_path_base+'/val_set/annotations/'

        self.img = img_load(self.img_path)
        self.label = label_load(self.label_path)
        self.transform = transform
        
    
    def __getitem__(self, idx):
        img_read = io.imread(self.img[idx]) 
        label_read = np.load(self.label[idx])
        label_read = label_read.astype(np.int64)
        label_tr = torch.from_numpy(label_read)
        
        
        if self.transform:
            img_tr = self.transform(img_read)
        
        
        return img_tr, label_tr
    

    def __len__(self):
        return len(self.label)
    
    if __name__ == "__main__":
        print('hi, i\'m main')

hi, i'm main


In [29]:
# Depthwise Saparable Convolution
class Depthwise(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.depthwise = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, stride=stride, padding=1, groups=in_channels, bias=False),
            # ==> nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
            # ==> kernel size = 3 이란 말은, 3x3을 의미함.
            nn.BatchNorm2d(in_channels),
            nn.ReLU6()
        )
        
        self.pointwise = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6()
        )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

In [30]:
Depthwise(32, 64)

Depthwise(
  (depthwise): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6()
  )
  (pointwise): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6()
  )
)

In [31]:
# Basic Conv2d
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.conv(x)
        return x

# Autograd + backward 연습

In [29]:
import torch
x = torch.tensor(data = [2.0, 3.0], requires_grad=True)
y = x**2
z = 2*y + 3

In [32]:
import torch
x = torch.tensor(data = [2.0, 3.0], requires_grad=True)
y = x**2
z = 2*y + 3

target = torch.tensor([3.0, 4.0])
loss = torch.sum(torch.abs(z-target))
print(z-target)
print(torch.abs(z-target))
print(loss)
loss_2 = torch.abs(z-target)
loss.backward()
print('loss : ',loss)

print('loss_2 : ', loss_2)

print(x.grad, y.grad, z.grad)

tensor([ 8., 17.], grad_fn=<SubBackward0>)
tensor([ 8., 17.], grad_fn=<AbsBackward0>)
tensor(25., grad_fn=<SumBackward0>)
loss :  tensor(25., grad_fn=<SumBackward0>)
loss_2 :  tensor([ 8., 17.], grad_fn=<AbsBackward0>)
tensor([ 8., 12.]) None None


In [33]:
for a in range(10):
    print(a)

print("b")

0
1
2
3
4
5
6
7
8
9
b
